# Scalable Generalized Linear Models 

Unlike linear regression, where the output is assumed to follow a Gaussian distribution, 
in [generalized linear models](https://en.wikipedia.org/wiki/Generalized_linear_model) (GLMs) the response variable $y$ follows some distribution from the [exponential family of distributions](https://en.wikipedia.org/wiki/Exponential_family).

In this Notebook we will look at Poisson regression over the [Bike Sharing Dataset](https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset). We will compare the performance of several models and algorithms on this dataset, including: Poisson Regression, Linear Regression implemented with IRLS and Linear Regression with general regularisation. 

In [1]:
# import findspark
# findspark.init()
from pyspark.sql import SparkSession
import numpy as np
spark = SparkSession.builder.master("local[2]").appName("COM6012 Generalized Linear Models").getOrCreate()

We now load the data. Here we use the hourly recordings only. 

In [2]:
rawdata = spark.read.csv('../Data/hour.csv', header=True)
rawdata.cache()

DataFrame[instant: string, dteday: string, season: string, yr: string, mnth: string, hr: string, holiday: string, weekday: string, workingday: string, weathersit: string, temp: string, atemp: string, hum: string, windspeed: string, casual: string, registered: string, cnt: string]

The following is a description of the features

    - instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered
    
From the above, we want to use the features season, yr, mnth, hr, holiday, weekday, workingday, weathersit, temp, atemp, hum and windspeed to predict cnt. 

In [3]:
schemaNames = rawdata.schema.names
ncolumns = len(rawdata.columns)
new_rawdata = rawdata.select(schemaNames[2:ncolumns])

Transform to DoubleType

In [4]:
new_schemaNames = new_rawdata.schema.names
from pyspark.sql.types import DoubleType
new_ncolumns = len(new_rawdata.columns)
for i in range(new_ncolumns):
    new_rawdata = new_rawdata.withColumn(new_schemaNames[i], new_rawdata[new_schemaNames[i]].cast(DoubleType()))

In [5]:
new_rawdata.printSchema()

root
 |-- season: double (nullable = true)
 |-- yr: double (nullable = true)
 |-- mnth: double (nullable = true)
 |-- hr: double (nullable = true)
 |-- holiday: double (nullable = true)
 |-- weekday: double (nullable = true)
 |-- workingday: double (nullable = true)
 |-- weathersit: double (nullable = true)
 |-- temp: double (nullable = true)
 |-- atemp: double (nullable = true)
 |-- hum: double (nullable = true)
 |-- windspeed: double (nullable = true)
 |-- casual: double (nullable = true)
 |-- registered: double (nullable = true)
 |-- cnt: double (nullable = true)



In [27]:
new_rawdata.columns

['season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

In [5]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = new_schemaNames[0:new_ncolumns-3], outputCol = 'features') 
data = assembler.transform(new_rawdata)

We now create the training and test data

In [7]:
(trainingData, testData) = data.randomSplit([0.7, 0.3], 42)
# (trainingData, testData) = ohe_data.randomSplit([0.7, 0.3], 42)

In [13]:
a = - (4/9) * np.log2(4/9) - (1/3) * np.log2(1/3) - (2/9) * np.log2(2/9)
a

1.5304930567574824

In [18]:
from pyspark.ml.feature import OneHotEncoderEstimator
feature = ['season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit']
ohe_feature = ['ohe_season','ohe_yr','ohe_mnth','ohe_hr','ohe_holiday','ohe_weekday','ohe_workingday','ohe_weathersit',
              ]

ohe = OneHotEncoderEstimator(inputCols= feature, outputCols=ohe_feature)
ohe_model = ohe.fit(trainingData)
ohe_model_test = ohe.fit(testData)
trainingData = ohe_model.transform(trainingData)
testData = ohe_model_test.transform(testData)

IllegalArgumentException: 'requirement failed: Column ohe_season already exists.'

In [19]:
assembler = VectorAssembler(inputCols = ohe_feature, outputCol = 'ohe_features_assembled') 
ohe_trainingData = assembler.transform(trainingData)
ohe_testData = assembler.transform(testData)

In [22]:
all_features = VectorAssembler(inputCols = ['ohe_features_assembled','temp','atemp','hum',
                                           'windspeed','workingday'], outputCol = 'new_feature')
trainingDataAll = all_features.transform(ohe_trainingData)
testDataAll = all_features.transform(ohe_testData)

We now want to proceed to apply Poisson Regression over our dataset. We will use the [GeneralizedLinearRegression](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html?highlight=generalized#pyspark.ml.regression.GeneralizedLinearRegression) model for which we can set the following parameters

> **maxIter**: max number of iterations.<p>
    **regParameter**: regularization parameter (>= 0). By setting this parameter to be >0 we are applying an $\ell_2$ regularization.<p>
**familiy**: The name of family which is a description of the error distribution to be used in the model. Supported options: gaussian (default), binomial, poisson, gamma and tweedie.<p>
    **link**: The name of link function which provides the relationship between the linear predictor and the mean of the distribution function. Supported options: identity, log, inverse, logit, probit, cloglog and sqrt. <p>
    The Table below shows the combinations of **family** and **link** functions allowed in this version of PySpark.<p>
        
<table>
<tr><td><b>Family</b></td><td><b>Response Type</b></td><td><b>Supported Links</b></td></tr>
<tr><td>Gaussian</td><td>Continuous</td><td>Identity, Log, Inverse</td></tr>
<tr><td>Binomial</td><td>Binary</td><td>Logit, Probit, CLogLog</td></tr>
<tr><td>Poisson</td><td>Count</td><td>Log, Identity, Sqrt</td></tr>
<tr><td>Gamma</td><td>Continuous</td><td>Inverse, Identity, Log</td></tr>
<tr><td>Tweedie</td><td>Zero-inflated continuous</td><td>Power link function</td></tr>
</table>    

In [31]:
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import LinearRegression 
#glm_poisson = GeneralizedLinearRegression(featuresCol='features', labelCol='cnt', maxIter=50, regParam=0.01,\
#                                          family='poisson', link='log')
glm_linear2  = LinearRegression(featuresCol='new_feature', labelCol='cnt', maxIter=50,regParam = 0.01, elasticNetParam = 0.5 )

#glm_model = glm_poisson.fit(trainingData)
linear_model2 = glm_linear2.fit(trainingDataAll)
#predictions = glm_model.transform(testData)
predictions2 = linear_model2.transform(testDataAll)

We now evaluate the RMSE

In [32]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator\
      (labelCol="cnt", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions2)
print("RMSE = %g " % rmse)

RMSE = 103.113 


In [10]:
glm_model.coefficients

DenseVector([0.1359, 0.4381, -0.0006, 0.0485, -0.0927, 0.0092, 0.0359, -0.073, 0.6825, 0.6532, -0.7603, 0.2205])

### Question 1

The variables season, yr, mnth, hr, holiday, weekday and weathersit are categorical variables that have been treated as continuous variables. In general this is not optimal since we are indirectly imposing a geometry or order over a variable that does not need to have such geometry. For example, the variable season takes values 1 (spring), 2 (summer), 3 (fall) and 4 (winter). Indirectly, we are saying that the distance between spring and winter (1 and 4) is larger than the distance between spring (1) and summer (3). There is not really a reason for this. To avoid this imposed geometries over variables that do not follow one, the usual approach is to transform categorical features to a representation of one-hot encoding. Use the [OneHotEncoderEstimator](http://spark.apache.org/docs/2.3.2/api/python/pyspark.ml.html#pyspark.ml.feature.OneHotEncoderEstimator) over the Bike Sharing Dataset to represent the categorical variables. Using the same training and test data compute the RMSE over the test data using the same Poisson model. 

### Question 2

Compare the performance of Linear Regression over the same dataset using the following algorithms: 

1. Linear Regression using $\ell_1$ regularisation and optimisation OWL-QN.
2. Linear Regression using elasticNet regularisation and optimisation OWL-QN.
3. Linear Regression using $\ell_2$ regularisation and optimisation L-BGFS.
4. Linear Regression using $\ell_2$ regularisation and optimisation IRLS.

### Question 3

The type of features used for regression can have a dramatic influence over the performance. When we use one-hot encoding for the categorical features, the prediction error of the Poisson regression reduces from 145 to 90 (see Question 1). We could further preprocess the features to see how the preprocessing can influence the performance. Test the performance of Poisson regression and the Linear Regression models in Question 2 when the continuous features are standardized (the mean of each feature is made equal to zero and the standard deviation is equal to one). Standardization is performed over the training data only, and the means and standard deviations computed over the training data are later used to standardize the test data. 